In [1]:
# Load Real OSM Roads Data

import geopandas as gpd
from shapely.geometry import box, LineString, Point
import pandas as pd
import os
import random

# Path to your real roads shapefile
roads_shapefile_path = r"c:\Users\sehga\Downloads\northern-zone-latest-free.shp\gis_osm_roads_free_1.shp"

# Check if roads shapefile exists (look for roads, not buildings)
if os.path.exists(roads_shapefile_path):
    print(f"✅ Found roads shapefile: {roads_shapefile_path}")
    try:
        # Load real roads data
        roads = gpd.read_file(roads_shapefile_path)
        print(f"🛣️ Loaded {len(roads)} road features")
        print(f"📊 Roads data columns: {list(roads.columns)}")
        print(f"🗺️ CRS: {roads.crs}")
        
        # Show first few road types if available
        if 'fclass' in roads.columns:
            print(f"🔍 Road types: {roads['fclass'].value_counts().head()}")
        elif 'highway' in roads.columns:
            print(f"🔍 Highway types: {roads['highway'].value_counts().head()}")
            
    except Exception as e:
        print(f"❌ Error loading roads: {e}")
        print("Creating synthetic road data instead...")
        roads = None
        
else:
    print(f"❌ Roads file not found at: {roads_shapefile_path}")
    print("📂 Available files in the directory:")
    
    # List available shapefiles
    downloads_dir = r"c:\Users\sehga\Downloads\northern-zone-latest-free.shp"
    if os.path.exists(downloads_dir):
        shp_files = [f for f in os.listdir(downloads_dir) if f.endswith('.shp')]
        for shp_file in shp_files:
            print(f"   📄 {shp_file}")
    
    roads = None

# Fallback to synthetic data if real roads not available
if roads is None:
    print("🔧 Creating synthetic road data for demonstration...")
    
    # Define bounding box coordinates (Northern India region)
    minx, miny, maxx, maxy = 77.0, 28.4, 77.3, 28.7  # Delhi area
    
    # Create synthetic road geometries
    roads_data = []
    
    # Generate some realistic road patterns
    for i in range(50):  # More roads for better training
        # Create road segments within the bbox
        x1 = random.uniform(minx, maxx)
        y1 = random.uniform(miny, maxy)
        x2 = random.uniform(minx, maxx)
        y2 = random.uniform(miny, maxy)
        
        road_line = LineString([(x1, y1), (x2, y2)])
        roads_data.append({
            'geometry': road_line, 
            'road_id': f'road_{i}',
            'fclass': random.choice(['primary', 'secondary', 'tertiary', 'residential'])
        })
    
    # Create GeoDataFrame
    roads = gpd.GeoDataFrame(roads_data, crs='EPSG:4326')
    print(f"✅ Created synthetic dataset with {len(roads)} road segments")

print(f"\n🎯 Final roads dataset: {len(roads)} features ready for processing")

✅ Found roads shapefile: c:\Users\sehga\Downloads\northern-zone-latest-free.shp\gis_osm_roads_free_1.shp
🛣️ Loaded 1797286 road features
📊 Roads data columns: ['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed', 'layer', 'bridge', 'tunnel', 'geometry']
🗺️ CRS: EPSG:4326
🔍 Road types: fclass
residential     1087620
service          247168
unclassified     150228
track            138666
tertiary          57700
Name: count, dtype: int64

🎯 Final roads dataset: 1797286 features ready for processing


In [2]:
# Enhanced Automation with Real Roads Data

import os
from pathlib import Path
from typing import List, Tuple
import geopandas as gpd
from shapely.geometry import box
from rasterio import features
from affine import Affine
import numpy as np
from PIL import Image

def get_tile_bounds(tile_id: str, center_coords: Tuple[float, float], 
                   tile_size: float = 0.01) -> box:
    """Generate bounding box for a tile"""
    lon, lat = center_coords
    half_size = tile_size / 2
    return box(lon - half_size, lat - half_size, 
              lon + half_size, lat + half_size)

def rasterize_and_save(clipped_roads: gpd.GeoDataFrame, 
                      tile_bounds: box, tile_output_mask_path: str, 
                      image_size: Tuple[int, int] = (256, 256)):
    """Rasterize roads and save mask - Your requested function"""
    height, width = image_size
    
    # Create affine transform
    minx, miny, maxx, maxy = tile_bounds.bounds
    transform = Affine.translation(minx, miny) * Affine.scale(
        (maxx - minx) / width, (maxy - miny) / height)
    
    # Rasterize roads with different weights for different road types
    if len(clipped_roads) > 0:
        # Create geometries with values (road importance)
        geometries = []
        for _, road in clipped_roads.iterrows():
            # Assign different values based on road type
            if 'fclass' in road and pd.notna(road['fclass']):
                road_type = road['fclass']
                if road_type in ['primary', 'trunk', 'motorway']:
                    value = 1  # Main roads
                elif road_type in ['secondary', 'tertiary']:
                    value = 1  # Secondary roads  
                else:
                    value = 1  # All other roads
            else:
                value = 1  # Default value
            
            geometries.append((road['geometry'], value))
        
        mask = features.rasterize(
            geometries,
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype=np.uint8
        )
    else:
        mask = np.zeros((height, width), dtype=np.uint8)
    
    # Create directory if needed
    os.makedirs(os.path.dirname(tile_output_mask_path), exist_ok=True)
    
    # Save mask
    mask_image = Image.fromarray(mask * 255)
    mask_image.save(tile_output_mask_path)
    
    return np.count_nonzero(mask)

def create_satellite_image_from_coords(tile_id: str, coords: Tuple[float, float], 
                                     output_path: str, image_size: Tuple[int, int] = (256, 256)):
    """Create more realistic satellite image based on coordinates"""
    height, width = image_size
    lon, lat = coords
    
    # Generate satellite-like imagery based on location
    # Base terrain (varies by location)
    base_color = np.random.randint(60, 120, (height, width, 3), dtype=np.uint8)
    
    # Add geographic features based on coordinates
    if 28.0 <= lat <= 29.0 and 77.0 <= lon <= 78.0:  # Delhi region
        # Urban area - more gray/concrete colors
        base_color[:, :, :] = np.random.randint(80, 140, (height, width, 3))
    elif 18.0 <= lat <= 19.0 and 73.0 <= lon <= 74.0:  # Pune region  
        # Mixed urban/rural - varied colors
        base_color[:, :, :] = np.random.randint(70, 130, (height, width, 3))
    
    # Add road-like features (linear structures)
    # Horizontal roads
    for i in range(0, height, 30):
        thickness = random.randint(2, 6)
        road_color = np.random.randint(90, 130, 3)
        base_color[i:i+thickness, :, :] = road_color
    
    # Vertical roads
    for j in range(0, width, 35):
        thickness = random.randint(2, 6)
        road_color = np.random.randint(90, 130, 3)
        base_color[:, j:j+thickness, :] = road_color
    
    # Add some noise for realism
    noise = np.random.randint(-20, 20, (height, width, 3))
    image = np.clip(base_color + noise, 0, 255).astype(np.uint8)
    
    # Create directory if needed
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save image
    Image.fromarray(image).save(output_path)
    
    return output_path

print("✅ Enhanced automation functions with real roads data ready!")

✅ Enhanced automation functions with real roads data ready!


In [3]:
# Main Automation Loop with Real Data

# Define satellite tiles based on your data region
# Adjust coordinates based on your northern zone data
satellite_tiles = [
    ("delhi_001", (77.2090, 28.6139)),   # Delhi center
    ("delhi_002", (77.2190, 28.6239)),   # Northeast Delhi
    ("delhi_003", (77.1990, 28.6039)),   # Southwest Delhi
    ("gurgaon_001", (77.0266, 28.4595)), # Gurgaon
    ("noida_001", (77.3910, 28.5355)),   # Noida
    ("faridabad_001", (77.3178, 28.4089)), # Faridabad
    ("pune_001", (73.8567, 18.5204)),    # Pune (if data covers this)
    ("mumbai_001", (72.8777, 19.0760)),  # Mumbai (if data covers this)
]

# Your requested automation loop with real roads data
results = []

print(f"🚀 Processing {len(satellite_tiles)} tiles with real OSM roads data...")
print(f"📊 Using roads dataset with {len(roads)} features")

for tile in satellite_tiles:
    tile_id, coords = tile
    print(f"\n📍 Processing {tile_id} at coordinates {coords}")
    
    # Get tile bounds (from georeferenced image)
    bounds = get_tile_bounds(tile_id, coords, tile_size=0.02)  # Larger tiles for more roads
    print(f"   Tile bounds: {bounds.bounds}")
    
    # Clip roads to tile bounds using real roads data
    clipped = roads[roads.intersects(bounds)]
    print(f"   Found {len(clipped)} real road segments")
    
    # Show road types in this tile
    if len(clipped) > 0 and 'fclass' in clipped.columns:
        road_types = clipped['fclass'].value_counts()
        print(f"   Road types: {dict(road_types.head(3))}")
    
    # Define output paths
    tile_output_mask_path = f"data/masks/{tile_id}_mask.png"
    tile_output_image_path = f"data/images/{tile_id}_image.png"
    
    # Create satellite image based on coordinates
    create_satellite_image_from_coords(tile_id, coords, tile_output_image_path)
    
    # Rasterize and save mask (your requested function)
    road_pixels = rasterize_and_save(clipped, bounds, tile_output_mask_path)
    
    # Store results
    results.append({
        'tile_id': tile_id,
        'coords': coords,
        'image_path': tile_output_image_path,
        'mask_path': tile_output_mask_path,
        'road_segments': len(clipped),
        'road_pixels': road_pixels,
        'has_roads': len(clipped) > 0
    })
    
    print(f"   ✅ Generated: {tile_id}_image.png & {tile_id}_mask.png")
    print(f"   📊 Road pixels: {road_pixels}")

print(f"\n🎉 Automation complete! Processed {len(results)} tiles with real OSM data")

# Filter results to show only tiles with roads
tiles_with_roads = [r for r in results if r['has_roads']]
print(f"🛣️ Tiles with roads: {len(tiles_with_roads)}/{len(results)}")

🚀 Processing 8 tiles with real OSM roads data...
📊 Using roads dataset with 1797286 features

📍 Processing delhi_001 at coordinates (77.209, 28.6139)
   Tile bounds: (77.199, 28.6039, 77.21900000000001, 28.623900000000003)
   Found 863 real road segments
   Road types: {'service': 360, 'secondary': 180, 'footway': 123}
   ✅ Generated: delhi_001_image.png & delhi_001_mask.png
   📊 Road pixels: 12021

📍 Processing delhi_002 at coordinates (77.219, 28.6239)
   Tile bounds: (77.20899999999999, 28.613899999999997, 77.229, 28.6339)
   Found 1502 real road segments
   Road types: {'service': 683, 'footway': 335, 'residential': 168}
   ✅ Generated: delhi_002_image.png & delhi_002_mask.png
   📊 Road pixels: 15188

📍 Processing delhi_003 at coordinates (77.199, 28.6039)
   Tile bounds: (77.189, 28.593899999999998, 77.209, 28.6139)
   Found 538 real road segments
   Road types: {'service': 226, 'secondary': 99, 'residential': 98}
   ✅ Generated: delhi_003_image.png & delhi_003_mask.png
   📊 Road 